# Create Video Thumbnail Table

Create a 3LC Table containing video files with automatically generated thumbnail images from the UCF11 action recognition dataset.

![img](../images/create-video-thumbnail-table.jpg)

<!-- Tags: ["video", "thumbnails", "action-recognition", "intermediate"] -->

Video datasets require visual previews for quick browsing and analysis. Thumbnails allow you to inspect video content at scale and are essential for video understanding tasks like action recognition, object tracking, and content moderation.

This notebook processes video files from the UCF11 dataset, automatically generating thumbnail images for each video clip. We use a TableWriter to create rows containing both video URLs and their corresponding thumbnail images. The dataset follows a structured format with categorized video clips:

```
UCF11/
├─ basketball/
│  ├─ v_shooting_01
|  │  ├─ v_shooting_01_01.mpg
|  │  ├─ v_shooting_01_02.mpg
|  │  ├─ ...
│  ├─ v_shooting_02
|  │  ├─ ...
├─ biking/
│  ├─ ...
├─ ...
```

Create a 3LC Table containing video files with automatically generated thumbnail images from the UCF11 action recognition dataset.

![img](../images/create-video-thumbnail-table.jpg)

<!-- Tags: ["video", "thumbnails", "action-recognition", "intermediate"] -->

Video datasets require visual previews for quick browsing and analysis. Thumbnails allow you to inspect video content at scale and are essential for video understanding tasks like action recognition, object tracking, and content moderation.

This notebook processes video files from the UCF11 dataset, automatically generating thumbnail images for each video clip. We use a TableWriter to create rows containing both video URLs and their corresponding thumbnail images. The dataset follows a structured format with categorized video clips:

```
UCF11/
├─ basketball/
│  ├─ v_shooting_01
|  │  ├─ v_shooting_01_01.mpg
|  │  ├─ v_shooting_01_02.mpg
|  │  ├─ ...
│  ├─ v_shooting_02
|  │  ├─ ...
├─ biking/
│  ├─ ...
├─ ...
```

## Project setup

In [ ]:
DATA_PATH = "../../data"
PROJECT_NAME = "3LC Tutorials - Create Tables"

## Imports

In [ ]:
from pathlib import Path

import cv2
import tlc

## Create Table

The class names are read from the directory names.

In [ ]:
DATASET_LOCATION = Path(DATA_PATH) / "ucf11"

assert DATASET_LOCATION.exists(), f"Dataset not found at {DATASET_LOCATION}"

In [ ]:
class_directories = [path for path in DATASET_LOCATION.glob("*") if path.is_dir()]

In [ ]:
classes = [c.name for c in class_directories]
classes

We now define a schema for the `Table`. Each row should consist of a generic `Url` to the video file, the thumbnail image and a categorical label for the video class.

In [ ]:
column_schemas = {
    "video_url": tlc.StringSchema("video_url"),
    "thumbnail": tlc.ImageUrlSchema(),
    "label": tlc.CategoricalLabelSchema(classes),
}

We then iterate over the videos, write the thumbnails next to the videos, and write the `Table` with a `TableWriter`.

In [ ]:
def create_thumbnail(video_path: Path, thumbnail_path: Path) -> None:
    """Read the first frame of a video and save it to disk.

    :param video_path: Path to the video file.
    :param thumbnail_path: Path to save the thumbnail image.
    """
    cap = cv2.VideoCapture(str(video_path))
    _, frame = cap.read()
    cv2.imwrite(str(thumbnail_path), frame)
    cap.release()

In [ ]:
table_writer = tlc.TableWriter(
    project_name=PROJECT_NAME,
    dataset_name="UCF YouTube Actions",
    table_name="initial",
    column_schemas=column_schemas,
)

for class_idx, class_directory in enumerate(class_directories):
    for video_path in class_directory.rglob("*mpg"):
        video_path = video_path.absolute()
        # Create a thumbnail
        thumbnail_path = video_path.with_suffix(".jpg")
        create_thumbnail(video_path, thumbnail_path)

        # Write the row
        row = {
            "video_url": tlc.Url(video_path).to_relative().to_str(),
            "thumbnail": tlc.Url(thumbnail_path).to_relative().to_str(),
            "label": class_idx,
        }

        table_writer.add_row(row)

table = table_writer.finalize()

In [ ]:
table